In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import yaml
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix, vstack
from scipy.sparse.linalg import svds
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import MinMaxScaler, normalize

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import ndcg_score, average_precision_score, precision_score, recall_score

import spotipy 
from spotipy.oauth2 import SpotifyOAuth
from surprise import NMF, SVD, SVDpp, KNNBasic, KNNWithMeans, KNNWithZScore, CoClustering
from surprise.model_selection import cross_validate
from surprise import Reader, Dataset

import scipy.sparse as sp

from surprise import Dataset
from surprise.model_selection import train_test_split

from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from mlxtend.evaluate import bias_variance_decomp

from tqdm import tqdm

import os
import sys
import surprise
import scrapbook as sb
import pandas as pd

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from recommenders.models.surprise.surprise_utils import predict, compute_ranking_predictions

from collections import defaultdict

from surprise import Dataset, SVD
from surprise.model_selection import KFold
from sklearn.model_selection import StratifiedKFold


from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

from typing import List
from math import sqrt

In [2]:
#POTREBNO IZMJENIT PUTANJU!!!!

#PREUZIMANJE PODATAKA GENERIRANIH IZ SPOTIFY

playlist_tracks_df_1 = pd.read_pickle(r"C:\Users\Ivan\PycharmProjects\sars4\sars4\spotify\playlist_tracks.pkl")
playlist_tracks_df_2 = pd.read_pickle(r"C:\Users\Ivan\PycharmProjects\sars8\spotify\playlist_tracks_2.pkl")
playlist_tracks_df_3 = pd.read_pickle(r"C:\Users\Ivan\PycharmProjects\sars10\spotify\playlist_tracks_3.pkl")
playlist_tracks_df = pd.concat([playlist_tracks_df_1,playlist_tracks_df_2, playlist_tracks_df_3], ignore_index=True)

In [3]:
playlist_tracks_df['event_strength'] = 1  

In [4]:
#DEFINIRANJE DATAFRAMEA GDJE JE DEFINIRAN SVAKI PAR (KORISNIK-STVAR)->OCJENA

playlist_tracks_df['event_strength'] = 0  
user_set = playlist_tracks_df['playlist_id'].unique()
item_set=playlist_tracks_df['id'].unique()
ptm = pd.DataFrame(columns=["userID", "itemID", "rating"])

In [5]:
#KOMBINIRAJ PAROVE KORISNIK-STVAR U DATAFRAME OBLIKA-(KORISNIK-STVAR->AKO JE KORISNIK IMAO INTERAKCIJU SA STVARI - 1, ELSE - 0)

ptm_dict={}
for user in tqdm(user_set):
    user_item_proto=playlist_tracks_df[playlist_tracks_df['playlist_id']==user]['id']
    
    ptm_dict[user]=pd.DataFrame(pd.Series({"userID": user, "itemID": item, "rating": 1}) 
                    if(user_item_proto.str.contains(item, case=False).any())
                    else (pd.Series({"userID": user, "itemID": item, "rating": 0}))
                    for item in item_set)

100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [22:45<00:00,  9.16s/it]


In [6]:
#SPREMI GENERIRANI DICTIONARY U ZAJEDNICKI DATAFRAME

ptm=pd.DataFrame(columns=["userID", "itemID", "rating"])

for elem in tqdm(ptm_dict):
    if len(ptm)==0:
        ptm=ptm_dict[elem]
    else:
        ptm = pd.concat([ptm, ptm_dict[elem]], ignore_index=True)


100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [00:02<00:00, 63.13it/s]


In [7]:
#DEFINIRAJ READER OD 0-1 S KOJIM CE SE PTM PRENIJET NA DATA
#DATA SE UČI NA ALGORITMU SVD()

reader = Reader(rating_scale=(0,1))
data = Dataset.load_from_df(ptm, reader)
algo = SVD()
algo.fit(data.build_full_trainset())
my_recs = []

In [9]:
#RADI PREDVIĐANJA

results_SVD_dict={}
for user in tqdm(user_set):
    results_SVD_dict[user]=pd.DataFrame(pd.Series({"userID": user, "itemID": item, "pred": algo.predict(uid=user,iid=item).est})
                                                for item in item_set)

100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [08:56<00:00,  3.60s/it]


In [10]:
#PRENESI PREDVIĐANJA IZ DICTIONARYJA U ZAJEDNIČKI DATAFRAME

results_SVD=pd.DataFrame(columns=["userID", "itemID", "rating"])
for elem in tqdm(results_SVD_dict):
    if len(results_SVD)==0:
        results_SVD=results_SVD_dict[elem]
    else:
        results_SVD = pd.concat([results_SVD, results_SVD_dict[elem]], ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████████████| 149/149 [00:02<00:00, 59.81it/s]


In [11]:
#SPAJANJE MATRICA USER-ITEM-RATING-PREDICTION

ui_rating_prediction_df = ptm.merge(results_SVD, on=['userID', 'itemID'], how='inner')

In [12]:
#IZRACUNAJ PROSJECNU VRIJEDNOST STOTOG ELEMENTA

average_100th_list=[]
for user in tqdm(results_SVD_dict):
    average_100th_list.append(results_SVD_dict[user]['pred'].sort_values(ascending=False).iloc[99])
average_100th=sum(average_100th_list)/len(average_100th_list)
print("AVERAGE 100TH:  ",average_100th)

100%|███████████████████████████████████████████████████████████████████████████████| 149/149 [00:00<00:00, 903.34it/s]

AVERAGE 100TH:   0.05117256954866526


In [13]:
#Izračunaj koliko se koja pjesma ponavlja u preporukama za sve korisnike

preds_dict_rec_100={}
items_count_dict={}
for item in playlist_tracks_df.index:
    items_count_dict[item]=0

for user in tqdm(results_SVD_dict):
    preds_dict_rec_100[user]=results_SVD_dict[user]['pred'].sort_values(ascending=False).head(99)
    for item in preds_dict_rec_100[user].index:
        items_count_dict[item]+=1

100%|███████████████████████████████████████████████████████████████████████████████| 149/149 [00:00<00:00, 745.24it/s]


In [14]:
print("CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-")

CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-CROSS VALIDATION-


In [24]:
average_rmse_cv1=cross_validate(algo, data, measures=['RMSE'], cv=10, verbose=True)
print('CV cross_validate RMSE: ' + str(sum(average_rmse_cv1['test_rmse']) / len(average_rmse_cv1['test_rmse'])))

Evaluating RMSE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.0994  0.1004  0.1017  0.1009  0.0968  0.0954  0.0979  0.0995  0.0974  0.0969  0.0986  0.0019  
Fit time          195.78  223.66  221.18  229.93  229.64  244.13  229.02  188.85  182.71  181.08  212.60  21.87   
Test time         17.09   3.92    2.40    5.08    4.09    3.47    3.89    3.66    2.00    3.20    4.88    4.15    
CV cross_validate RMSE: 0.09863147379324289


In [25]:
average_mae_cv1=cross_validate(algo, data, measures=['MAE'], cv=10, verbose=True)
print('CV cross_validate MAE: ' + str(sum(average_mae_cv1['test_mae']) / len(average_mae_cv1['test_mae'])))

Evaluating MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
MAE (testset)     0.0195  0.0197  0.0200  0.0202  0.0200  0.0199  0.0205  0.0196  0.0196  0.0197  0.0199  0.0003  
Fit time          175.31  122.38  120.84  111.04  103.96  108.99  106.06  107.31  106.15  108.93  117.10  20.27   
Test time         1.73    2.05    2.35    1.20    1.73    1.83    1.20    1.88    1.17    1.15    1.63    0.40    
CV cross_validate MAE: 0.019879080528878313


In [15]:
kf = KFold(n_splits=10)
algo_test = SVD()
rmse_results_cv2=[]
mae_results_cv2=[]
precision_results_cv_micro=[]
precision_results_cv_macro=[]
precision_results_cv_weighted=[]
recall_results_cv_micro=[]
recall_results_cv_macro=[]
recall_results_cv_weighted=[]

map_results_cv = []
ndcg_results_cv = []

for trainset, testset in tqdm(kf.split(data)):
    algo_test.fit(trainset)                      #NAUCI NA TRAINSETU         
    predictions = algo_test.test(testset)        #NAPRAVI PREDVIDANJA NA TESTSETU
    
    dframe = pd.DataFrame(predictions, columns=["uid", "iid", "true_r", "est", "details"])   #STAVI U ISTI DATASET
    
    #EVALUACIJA
    
    rmse_results_cv2.append(sqrt(mean_squared_error(dframe['est'], dframe['true_r'])))     
    mae_results_cv2.append((mean_absolute_error(dframe['est'], dframe['true_r'])))
    
    average_map = average_precision_score(dframe['true_r'].values, dframe['est'].values)
    map_results_cv.append(average_map)
        
    average_ndcg=ndcg_score(dframe['true_r'].values.reshape(1, -1), dframe['est'].values.reshape(1, -1))
    ndcg_results_cv.append(average_ndcg)
    
    df=dframe.copy()
    df.loc[df['est']>=average_100th, 'est']=1
    df.loc[df['est']<average_100th, 'est']=0

    precision_results_cv_micro.append(precision_score(df['true_r'], df['est'], average='micro'))
    recall_results_cv_micro.append(recall_score(df['true_r'], df['est'], average='micro'))
    
    precision_results_cv_macro.append(precision_score(df['true_r'], df['est'], average='macro'))
    recall_results_cv_macro.append(recall_score(df['true_r'], df['est'], average='macro'))
    
    precision_results_cv_weighted.append(precision_score(df['true_r'], df['est'], average='weighted'))
    recall_results_cv_weighted.append(recall_score(df['true_r'], df['est'], average='weighted'))
    


10it [18:15, 109.56s/it]


In [16]:
mean_rmse_score_cv=np.mean(rmse_results_cv2)
mean_mae_score_cv=np.mean(mae_results_cv2)
mean_precision_score_cv_micro=np.mean(precision_results_cv_micro)
mean_recall_score_cv_micro=np.mean(recall_results_cv_micro)
mean_precision_score_cv_macro=np.mean(precision_results_cv_macro)
mean_recall_score_cv_macro=np.mean(recall_results_cv_macro)
mean_precision_score_cv_weighted=np.mean(precision_results_cv_weighted)
mean_recall_score_cv_weighted=np.mean(recall_results_cv_weighted)
mean_map_score_cv = np.mean(map_results_cv)
mean_ndcg_score_cv = np.mean(ndcg_results_cv)
f1_score_cv_micro=(2*mean_precision_score_cv_micro*mean_recall_score_cv_micro)/(mean_precision_score_cv_micro+mean_recall_score_cv_micro)
f1_score_cv_macro=(2*mean_precision_score_cv_macro*mean_recall_score_cv_macro)/(mean_precision_score_cv_macro+mean_recall_score_cv_macro)
f1_score_cv_weighted=(2*mean_precision_score_cv_weighted*mean_recall_score_cv_weighted)/(mean_precision_score_cv_weighted+mean_recall_score_cv_weighted)

print("CV SKLEARN.METRICS RMSE SCORE:    ",mean_rmse_score_cv)
print("CV SKLEARN.METRICS MAE SCORE:    ",mean_mae_score_cv)

print("CV SKLEARN.METRICS MAP SCORE:    ", mean_map_score_cv)
print("CV SKLEARN.METRICS NDCG SCORE:    ", mean_ndcg_score_cv)

print("CV SKLEARN.METRICS PRECISION SCORE:",'micro:  ',mean_precision_score_cv_micro, ', macro:  ',
                      mean_precision_score_cv_macro,',weighted:   ',mean_precision_score_cv_weighted)

print("CV SKLEARN.METRICS RECALL SCORE:    ",'micro:  ', mean_recall_score_cv_micro, ', macro:  ',
                              mean_recall_score_cv_macro,',weighted:   ', mean_recall_score_cv_weighted)

print("CV SKLEARN.METRICS F1:   ", f1_score_cv_micro,"|",
                      f1_score_cv_macro, "|", f1_score_cv_weighted)


CV SKLEARN.METRICS RMSE SCORE:     0.09865669108115671
CV SKLEARN.METRICS MAE SCORE:     0.019971402832553723
CV SKLEARN.METRICS MAP SCORE:     0.038646456824834495
CV SKLEARN.METRICS NDCG SCORE:     0.6205682811164819
CV SKLEARN.METRICS PRECISION SCORE: micro:   0.9700038056264717 , macro:   0.5328528816423982 ,weighted:    0.9825176835246895
CV SKLEARN.METRICS RECALL SCORE:     micro:   0.9700038056264717 , macro:   0.577007820249823 ,weighted:    0.9700038056264717
CV SKLEARN.METRICS F1:    0.9700038056264717 | 0.5540520161244067 | 0.9762206433165364


In [17]:
print("TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-")

TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-TEST-TRAIN-


In [18]:
trainset_tt, testset_tt = train_test_split(data, test_size=0.2, random_state=42)

#PONAVLJANJE ISTOG PROCESA KAO NA CV SKLEARN.METRICS

algo_tt = SVD()
algo_tt.fit(trainset_tt)
predictions_tt = algo_tt.test(testset_tt)

dframe_tt = pd.DataFrame(predictions_tt, columns=["uid", "iid", "true_r", "est", "details"])

#EVALUACIJA

average_rmse_TT=sqrt(mean_squared_error(dframe_tt['est'], dframe_tt['true_r']))
average_mae_TT=(mean_absolute_error(dframe_tt['est'], dframe_tt['true_r']))
average_ndcg=ndcg_score(dframe_tt['true_r'].values.reshape(1, -1), dframe_tt['est'].values.reshape(1, -1))
average_map=average_precision_score(dframe_tt['true_r'].values, dframe_tt['est'].values)

df=dframe_tt.copy()
df.loc[df['est']>=average_100th, 'est']=1
df.loc[df['est']<average_100th, 'est']=0

average_precision_micro=(precision_score(df['true_r'], df['est'], average='micro'))
average_recall_micro=(recall_score(df['true_r'], df['est'], average='micro'))
average_precision_macro=(precision_score(df['true_r'], df['est'], average='macro'))
average_recall_macro=(recall_score(df['true_r'], df['est'], average='macro'))
average_precision_weighted=(precision_score(df['true_r'], df['est'], average='weighted'))
average_recall_weighted=(recall_score(df['true_r'], df['est'], average='weighted'))
f1_score_micro=(2*average_precision_micro*average_recall_micro)/(average_precision_micro+average_recall_micro)
f1_score_macro=(2*average_precision_macro*average_recall_macro)/(average_precision_macro+average_recall_macro)
f1_score_weighted=(2*average_precision_weighted*average_recall_weighted)/(average_precision_weighted+average_recall_weighted)


print('TRAIN-TEST SKLEARN.METRICS RMSE:   ', average_rmse_TT)
print('TRAIN-TEST SKLEARN.METRICS MAE: ' + str(average_mae_TT))
print('TRAIN-TEST SKLEARN.METRICS NDCG: ' + str(average_ndcg))
print('TRAIN-TEST SKLEARN.METRICS MAP: ' + str(average_map))
print("TRAIN-TEST SKLEARN.METRICS PRECISION SCORE: ",average_precision_micro,
      average_precision_macro, average_precision_weighted)
print("TRAIN-TEST SKLEARN.METRICS RECALL SCORE:   ",average_recall_micro, average_recall_macro, average_recall_weighted) 
print("TRAIN-TEST SKLEARN.METRICS F1:   ", f1_score_micro,"|",
                      f1_score_macro, "|", f1_score_weighted)


TRAIN-TEST SKLEARN.METRICS RMSE:    0.09912246104351213
TRAIN-TEST SKLEARN.METRICS MAE: 0.020140474545661164
TRAIN-TEST SKLEARN.METRICS NDCG: 0.6466572296559775
TRAIN-TEST SKLEARN.METRICS MAP: 0.03754100814274393
TRAIN-TEST SKLEARN.METRICS PRECISION SCORE:  0.9698571895781096 0.5313939478476333 0.9822545685792448
TRAIN-TEST SKLEARN.METRICS RECALL SCORE:    0.9698571895781096 0.5726490810513785 0.9698571895781096
TRAIN-TEST SKLEARN.METRICS F1:    0.9698571895781096 | 0.5512507174918203 | 0.9760165127346512


In [19]:
print("NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION-NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION")

NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION-NOVELTY-CATALOG COVERAGE-PREDICTION COVERAGE-PERSONALIZATION


In [20]:
#RECMETRICS NOVELTY
def novelty(predicted: dict, pop: dict, u: int, n: int) -> (float, list):
    
    mean_self_information = []
    k = 0
    for pred100 in predicted: 
        self_information = 0
        k += 1
        for track_index in predicted[pred100].index:
            self_information += np.sum(-np.log2(pop[track_index]/u))
        mean_self_information.append(self_information/n)
    novelty = sum(mean_self_information)/k
    return novelty, mean_self_information
    
test_users = len(playlist_tracks_df) 
test_recs_per_user = 100

novelty_score, _ = novelty(
            predicted = preds_dict_rec_100,#predicted data
            pop = items_count_dict, #test data
            u = test_users, #broj korisnika
            n = test_recs_per_user #broj testnih recorda
)

print("NOVELTY SCORE:   ",novelty_score)

NOVELTY SCORE:    6.131760118408787


In [21]:
#RECMETRICS CATALOG COVERAGE

def catalog_coverage(catalog: dict) -> float:
    sum_non_zero=0
    for elem in catalog:
        if(catalog[elem]!=0):
            sum_non_zero+=1
    catalog_coverage = round(sum_non_zero/(len(catalog)*1.0),2)
    return catalog_coverage
catalog_coverage = catalog_coverage(
    catalog = items_count_dict, #test data
)

print("CATALOG COVERAGE SCORE:   ",catalog_coverage)


CATALOG COVERAGE SCORE:    0.01


In [22]:
#RECMETRICS PERSONALIZATION

def personalization(predicted: List[list]) -> float:

    def make_rec_matrix(predicted: List[list]) -> sp.csr_matrix:
        df = pd.DataFrame(data=predicted).reset_index().melt(
            id_vars='index', value_name='item',
        )
        df = df[['index', 'item']].pivot(index='index', columns='item', values='item')
        df = pd.notna(df)*1
        rec_matrix = sp.csr_matrix(df.values)
        return rec_matrix

    #create matrix for recommendations
    predicted = np.array(predicted)
    rec_matrix_sparse = make_rec_matrix(predicted)

    #calculate similarity for every user's recommendation list
    similarity = cosine_similarity(X=rec_matrix_sparse, dense_output=False)

    #calculate average similarity
    dim = similarity.shape[0]
    personalization = (similarity.sum() - dim) / (dim * (dim - 1))
    return 1-personalization

pred100_list = [preds_dict_rec_100[playlist].index.tolist() for playlist in preds_dict_rec_100]

# WHEN metrics.personalization is run
personalization_score =personalization(
    predicted=pred100_list)

# THEN the personalization score should be within the expected value
print("PERSONALIZATION SCORE:    ",personalization_score)

PERSONALIZATION SCORE:     0.04328703321992067


In [23]:
#RECMETRICS DIVERSITY
#PRESJEK/UNIJA

def jaccard_similarity(dframe, average_100th):
    index_rating = dframe[dframe['rating']==0].index
    index_pred = dframe[dframe['pred']>average_100th].index
    intersection_size = len(index_rating[index_rating.isin(index_pred)])
    union_size = len(index_rating.union(index_pred))
    return intersection_size / union_size if union_size != 0 else 0.0

def inter_list_diversity(dframe, average_100th):
    length_frame = len(dframe)
    pairwise_similarities = []    
    similarity = jaccard_similarity(dframe, average_100th)    
    return similarity

diversity_score = inter_list_diversity(ui_rating_prediction_df,average_100th)
print("Inter-List Diversity Score:", diversity_score)


Inter-List Diversity Score: 0.06154516282433993
